In [42]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [43]:
exp_2022 = pd.read_csv("..\Dados\COMEX\EXP_2022_MUN.csv", delimiter=";")
agropecuaria = pd.read_excel(r"..\Dados\COMEX\sh4_agropecuaria.xlsx")
tabela_final = pd.read_csv(r"..\Dados\Tabela_final\tabela_final.csv")
municipio_escolhidos = pd.read_csv(r"..\Dados\Tabela_final\municipios_potenciais.csv", index_col=0)

In [44]:
exp_2022

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB
0,2022,4,9403,845,PR,4101002,53482,47615
1,2022,10,8501,386,SC,4208906,220899,1348091
2,2022,9,1902,580,ES,3205309,564,2264
3,2022,1,6206,63,SC,4216602,85,6642
4,2022,2,8501,249,SC,4208906,2942192,12155328
...,...,...,...,...,...,...,...,...
1087126,2022,9,2201,445,SP,3448500,1260,328
1087127,2022,7,9603,586,RS,4304804,24,350
1087128,2022,9,8527,239,PR,4120804,234,34731
1087129,2022,1,203,767,PA,1506807,30,226


In [45]:
exp_2022 = exp_2022.merge(agropecuaria, how="inner", left_on="SH4", right_on="Código SH4")
exp_2022.drop(columns=["Código SH4"], inplace=True)

In [46]:
exp_2022

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,Descrição do produto exportado,Denominação nesse estudo
0,2022,9,5201,455,MT,5107875,1114253,2071800,"Algodão, não cardado nem penteado.",Algodão
1,2022,2,5201,827,MT,5107875,5938815,10428103,"Algodão, não cardado nem penteado.",Algodão
2,2022,2,5201,365,MT,5103502,1047269,1719167,"Algodão, não cardado nem penteado.",Algodão
3,2022,11,5201,858,MT,5107875,4691205,8303458,"Algodão, não cardado nem penteado.",Algodão
4,2022,3,5201,160,MA,2101400,170943,367305,"Algodão, não cardado nem penteado.",Algodão
...,...,...,...,...,...,...,...,...,...,...
121579,2022,8,4812,580,SP,3452205,2,178,"Blocos e chapas, filtrantes, de pasta de papel.",Papel
121580,2022,11,4812,493,SP,3430706,460,3989,"Blocos e chapas, filtrantes, de pasta de papel.",Papel
121581,2022,11,4812,850,SP,3443402,13,4616,"Blocos e chapas, filtrantes, de pasta de papel.",Papel
121582,2022,12,4812,43,ES,3205309,750,8000,"Blocos e chapas, filtrantes, de pasta de papel.",Papel


In [47]:
exp_2022 = exp_2022.groupby(["CO_MUN", "Denominação nesse estudo"]).agg({"VL_FOB":"mean", "KG_LIQUIDO":"mean"}).reset_index()

In [48]:
exp_2022_sertao = exp_2022[exp_2022["CO_MUN"].isin(tabela_final["IBGE7"])]

exp_2022_sertao.set_index("CO_MUN", inplace=True)
municipio_escolhidos.set_index("IBGE7", inplace=True)
tabela_final.set_index("IBGE7", inplace=True)

In [32]:
exp_2022.head()

,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
0,1100023,Milho/Soja em grãos,4.874950e+05,7.500000e+05
1,1100023,Soja em grãos,4.874950e+05,7.500000e+05
2,1100049,Café em grãos,5.645500e+04,2.426767e+04
3,1100049,Carne de Frango e Miudezas,1.206043e+05,7.466000e+04
4,1100049,Milho em grãos,2.113112e+06,9.990010e+06


In [31]:
municipio_escolhidos.head()

,UF,NOME,LATITUDE,LONGITUDE,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,...,T_ANALF18A24,T_ANALF18M,T_ANALF25A29,T_ANALF25M,QUAL_MED_AGUA,DIST_CORPO_AGUA,TRANSPORT_COST,TOT_ESC_POR_POP,TOT_MAT_POR_POP,CLUSTER
IBGE7,,,,,,,,,,,,,,,,,,,,,
2916500,BA,ITAPICURU,-11.30880,-38.2262,39.498593,50.529793,4.029523,2.864316,24.86,26.16,...,15.55,41.92,26.12,48.52,65.417601,3.461062,23766.055025,0.000752,0.221999,2
2705002,AL,MATA GRANDE,-9.11824,-37.7323,37.369491,43.018642,4.248078,3.359025,24.36,37.06,...,15.72,42.12,29.64,49.44,74.933851,61.185634,24784.146031,0.001785,0.217440,2
2921500,BA,MONTE SANTO,-10.43740,-39.3321,41.109244,48.624520,3.960775,3.141736,21.18,31.52,...,9.95,38.64,20.57,45.32,71.624686,5.415886,24661.745267,0.001481,0.251471,2
2607505,PE,ITAÍBA,-8.94569,-37.4173,32.710603,40.005640,4.135429,3.314976,38.30,38.98,...,13.54,42.95,26.51,50.72,66.893107,57.996168,24766.124221,0.001292,0.253649,2
2615805,PE,TUPANATINGA,-8.74798,-37.3445,30.184538,37.498476,5.175386,3.625949,31.38,50.88,...,13.65,41.81,24.21,49.72,66.097240,44.602110,24850.504059,0.001053,0.207216,2


In [75]:
def distancia_circular(df_escolhidos, df_municipios_sertao, threhold):
    dicionario_distancias = {}
    lista_distancias = []
    for municipio_escolhido, row in df_escolhidos[["LATITUDE", "LONGITUDE"]].iterrows():
        for municipio_qualquer, row2 in df_municipios_sertao[["LATITUDE", "LONGITUDE"]].iterrows():
            if municipio_escolhido != municipio_qualquer:
                distancia = geodesic(row.values, row2.values).kilometers
                if distancia <= threhold:
                    lista_distancias.append((municipio_qualquer, distancia))
    
        dicionario_distancias[df_escolhidos.loc[municipio_escolhido, "NOME"]] = lista_distancias
        lista_distancias = []

    return dicionario_distancias

In [77]:
distancias_municipios_escolhidos = distancia_circular(municipio_escolhidos, tabela_final, 50)

In [81]:
distancias_municipios_escolhidos

{'ITAPICURU': [(2907905, 39.14188507636838),
  (2909604, 23.27705027987044),
  (2922904, 29.69824588083892),
  (2923100, 13.005936550529988),
  (2926509, 36.578826160540316),
  (2807402, 28.573106808676275)],
 'MATA GRANDE': [(2701605, 14.90588369308685),
  (2702405, 41.58842418772786),
  (2703304, 12.086025672069255),
  (2704609, 43.55828891872997),
  (2705804, 43.9523160534112),
  (2706109, 41.648070520229396),
  (2706422, 33.03213021230852),
  (2708956, 49.22728937071549),
  (2607000, 26.447043603248208),
  (2607505, 39.54369978105547),
  (2609154, 20.271648554067216),
  (2614808, 46.01428721830516)],
 'MONTE SANTO': [(2906808, 30.784209870479184),
  (2910701, 35.545819229170554),
  (2922656, 43.56109186028162),
  (2925907, 36.97699954556055)],
 'ITAÍBA': [(2701605, 27.529036722730993),
  (2703304, 48.01762217634919),
  (2704609, 32.29300068158176),
  (2705002, 39.54369978105547),
  (2706109, 24.53129644730895),
  (2707206, 42.424593143362245),
  (2600500, 37.2181936423022),
  (2602

In [139]:
def produtos_exportados_vizinhos(dicionario, expor):
    dicionario_exportacao = {}
    lista_exportacao = []
    contagem = 0
    
    for key, value in dicionario.items():
        for municipio_vizinho in value:
            if  expor.isin([municipio_vizinho[0]]).any().any():
                contagem += 1
                produto = expor[expor["CO_MUN"] == municipio_vizinho[0]]["Denominação nesse estudo"].values
                kg_liq = expor[expor["CO_MUN"] == municipio_vizinho[0]]["KG_LIQUIDO"].values
                valor_fb = expor[expor["CO_MUN"] == municipio_vizinho[0]]["VL_FOB"].values
                #cod_pais = expor.loc[municipio_vizinho, "VL_FOB"]
                
                lista_exportacao.append((municipio_vizinho[0], produto, kg_liq, valor_fb))
                display(expor[expor["CO_MUN"] == municipio_vizinho[0]])

        if contagem:
            print(f"{key}\n")
            dicionario_exportacao[key] = lista_exportacao
        contagem = 0
        lista_exportacao = []
        
    return dicionario_exportacao

In [141]:
dicionario_export = produtos_exportados_vizinhos(distancias_municipios_escolhidos, exp_2022)

,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
687,2922904,Sumos de frutas (incluídos os mostos de uvas) ...,248108.545455,82893.25


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
610,2807402,Coprodutos do Algodão,22545.0,1526.0


ITAPICURU



,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
530,2606002,Coprodutos do milho,29500.8,33765.2
531,2606002,Farelo/Farinha e Resíduos Sólidos,29500.8,33765.2


CAETÉS



...

,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
504,2504009,Papel,1840.5,416.0


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
505,2506301,Carne de Frango e Miudezas,34305.44,57228.0


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
510,2512408,Papel,1351.0,109.0


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
513,2515104,Algodão,60623.0,11565.0


ALAGOA GRANDE

